In [10]:
# ============================================
# 1. IMPORTS
# ============================================
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import ta

from ipywidgets import interact, Dropdown

# ============================================
# 2. HELPER: CONVERT ANYTHING → SINGLE FLOAT
# ============================================

def to_scalar(x):
    """
    Safely convert Series / array / list / scalar to a single float.
    """
    if isinstance(x, (pd.Series, pd.DataFrame, np.ndarray, list, tuple)):
        return float(np.array(x).flatten()[0])
    return float(x)

# ============================================
# 3. NIFTY 50 MAP (NAME → TICKER)
# ============================================

nifty50_map = {
    "ADANIENT": {"ticker": "ADANIENT.NS"},
    "ADANIPORTS": {"ticker": "ADANIPORTS.NS"},
    "APOLLOHOSP": {"ticker": "APOLLOHOSP.NS"},
    "ASIANPAINT": {"ticker": "ASIANPAINT.NS"},
    "AXISBANK": {"ticker": "AXISBANK.NS"},
    "BAJAJ-AUTO": {"ticker": "BAJAJ-AUTO.NS"},
    "BAJAJFINSV": {"ticker": "BAJAJFINSV.NS"},
    "BAJFINANCE": {"ticker": "BAJFINANCE.NS"},
    "BPCL": {"ticker": "BPCL.NS"},
    "BHARTIARTL": {"ticker": "BHARTIARTL.NS"},
    "BRITANNIA": {"ticker": "BRITANNIA.NS"},
    "CIPLA": {"ticker": "CIPLA.NS"},
    "COALINDIA": {"ticker": "COALINDIA.NS"},
    "DIVISLAB": {"ticker": "DIVISLAB.NS"},
    "DRREDDY": {"ticker": "DRREDDY.NS"},
    "EICHERMOT": {"ticker": "EICHERMOT.NS"},
    "GRASIM": {"ticker": "GRASIM.NS"},
    "HCLTECH": {"ticker": "HCLTECH.NS"},
    "HDFCBANK": {"ticker": "HDFCBANK.NS"},
    "HDFCLIFE": {"ticker": "HDFCLIFE.NS"},
    "HEROMOTOCO": {"ticker": "HEROMOTOCO.NS"},
    "HINDALCO": {"ticker": "HINDALCO.NS"},
    "HINDUNILVR": {"ticker": "HINDUNILVR.NS"},
    "ICICIBANK": {"ticker": "ICICIBANK.NS"},
    "ITC": {"ticker": "ITC.NS"},
    "INFY": {"ticker": "INFY.NS"},
    "INDUSINDBK": {"ticker": "INDUSINDBK.NS"},
    "JSWSTEEL": {"ticker": "JSWSTEEL.NS"},
    "KOTAKBANK": {"ticker": "KOTAKBANK.NS"},
    "LT": {"ticker": "LT.NS"},
    "M&M": {"ticker": "M&M.NS"},
    "MARUTI": {"ticker": "MARUTI.NS"},
    "NESTLEIND": {"ticker": "NESTLEIND.NS"},
    "NTPC": {"ticker": "NTPC.NS"},
    "ONGC": {"ticker": "ONGC.NS"},
    "POWERGRID": {"ticker": "POWERGRID.NS"},
    "RELIANCE": {"ticker": "RELIANCE.NS"},
    "SBIN": {"ticker": "SBIN.NS"},
    "SHREECEM": {"ticker": "SHREECEM.NS"},
    "SUNPHARMA": {"ticker": "SUNPHARMA.NS"},
    "TATACONSUM": {"ticker": "TATACONSUM.NS"},
    "TATAMOTORS": {"ticker": "TATAMOTORS.NS"},
    "TATASTEEL": {"ticker": "TATASTEEL.NS"},
    "TCS": {"ticker": "TCS.NS"},
    "TECHM": {"ticker": "TECHM.NS"},
    "TITAN": {"ticker": "TITAN.NS"},
    "ULTRACEMCO": {"ticker": "ULTRACEMCO.NS"},
    "UPL": {"ticker": "UPL.NS"},
    "WIPRO": {"ticker": "WIPRO.NS"},
}

# ============================================
# 4. PRICE DATA – DAILY HISTORY (LATEST AVAILABLE)
# ============================================

def get_price_data(ticker, years=2):
    """
    Downloads daily price data for the last `years` using yfinance.
    Uses 'period' so the latest available day is included.
    """
    period_str = f"{years}y"

    df = yf.download(
        ticker,
        period=period_str,
        interval="1d",
        auto_adjust=False,
        progress=False
    )

    # Flatten possible MultiIndex columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df.dropna(inplace=True)
    return df


def add_technical_indicators(df):
    """
    Adds 20, 50, 200 day moving averages and 14-day RSI.
    """
    close = df['Close']
    if isinstance(close, pd.DataFrame):
        close = close.iloc[:, 0]

    df['MA20'] = close.rolling(window=20).mean()
    df['MA50'] = close.rolling(window=50).mean()
    df['MA200'] = close.rolling(window=200).mean()

    df['RSI14'] = ta.momentum.rsi(close, window=14)
    df.dropna(inplace=True)
    return df

# ============================================
# 5. LIVE / NEAR-LIVE PRICE
# ============================================

def get_live_price(ticker):
    """
    Gets the most recent available price from yfinance.
    Usually near-live but may be delayed ~15 minutes.
    """
    try:
        tk = yf.Ticker(ticker)
        price = None

        fast = getattr(tk, "fast_info", None)
        if fast is not None:
            price = fast.get("last_price", None)

        if price is None:
            info = tk.info
            price = info.get("currentPrice", None)

        return price
    except Exception as e:
        print(f"⚠️ Could not fetch live price: {e}")
        return None

# ============================================
# 6. FUNDAMENTALS FROM YFINANCE
# ============================================

def get_fundamentals_yfinance(ticker):
    """
    Gets basic fundamentals using yfinance's .info:
    PE, ROE, Debt/Equity.
    """
    try:
        tk = yf.Ticker(ticker)
        info = tk.info
    except Exception as e:
        print(f"⚠️ Could not fetch fundamentals from yfinance: {e}")
        return {"pe": None, "roe": None, "de_ratio": None}

    pe = info.get("trailingPE")
    roe = info.get("returnOnEquity")   # often decimal like 0.18
    de_ratio = info.get("debtToEquity")

    if roe is not None:
        roe = roe * 100   # 0.18 → 18%

    return {
        "pe": pe,
        "roe": roe,
        "de_ratio": de_ratio
    }

# ============================================
# 7. SCORING FUNCTIONS
# ============================================

def score_technicals(df):
    """
    Scores trend and momentum based on last row.
    """
    latest = df.iloc[-1]

    price = to_scalar(latest['Close'])
    ma20 = to_scalar(latest['MA20'])
    ma50 = to_scalar(latest['MA50'])
    ma200 = to_scalar(latest['MA200'])
    rsi = to_scalar(latest['RSI14'])

    score = 0
    reasons = []

    # Trend
    if price > ma20 > ma50 > ma200:
        score += 2
        reasons.append("Strong uptrend (Price > MA20 > MA50 > MA200)")
    elif price > ma50 > ma200:
        score += 1
        reasons.append("Uptrend (Price > MA50 > MA200)")
    elif price < ma200:
        score -= 1
        reasons.append("Price below 200-day MA (weak trend)")
    else:
        reasons.append("Neutral trend")

    # Momentum (RSI)
    if 50 <= rsi <= 70:
        score += 1
        reasons.append(f"Positive momentum (RSI={rsi:.1f})")
    elif rsi < 40:
        score -= 1
        reasons.append(f"Weak momentum / oversold (RSI={rsi:.1f})")
    else:
        reasons.append(f"Neutral momentum (RSI={rsi:.1f})")

    return score, reasons


def score_fundamentals(f):
    """
    Scores valuation and quality using PE, ROE, Debt/Equity.
    """
    score = 0
    reasons = []

    pe = f.get("pe")
    roe = f.get("roe")
    de = f.get("de_ratio")

    # PE
    if pe is not None:
        if pe <= 20:
            score += 1
            reasons.append(f"Attractive valuation (PE={pe:.1f} ≤ 20)")
        elif pe > 35:
            score -= 1
            reasons.append(f"Expensive valuation (PE={pe:.1f} > 35)")
        else:
            reasons.append(f"Reasonable valuation (PE={pe:.1f})")
    else:
        reasons.append("PE not available")

    # ROE
    if roe is not None:
        if roe >= 15:
            score += 1
            reasons.append(f"Good profitability (ROE={roe:.1f}% ≥ 15%)")
        elif roe < 10:
            score -= 1
            reasons.append(f"Weak profitability (ROE={roe:.1f}% < 10%)")
        else:
            reasons.append(f"Average profitability (ROE={roe:.1f}%)")
    else:
        reasons.append("ROE not available")

    # D/E
    if de is not None:
        if de <= 0.5:
            score += 1
            reasons.append(f"Low leverage (Debt/Equity={de:.2f} ≤ 0.5)")
        elif de > 1:
            score -= 1
            reasons.append(f"High leverage (Debt/Equity={de:.2f} > 1)")
        else:
            reasons.append(f"Moderate leverage (Debt/Equity={de:.2f})")
    else:
        reasons.append("Debt/Equity not available")

    return score, reasons


def final_recommendation(tech_score, fund_score):
    total = tech_score + fund_score

    if total >= 3:
        rec = "BUY"
    elif total <= 0:
        rec = "SELL"
    else:
        rec = "HOLD"

    return rec, total

# ============================================
# 8. MAIN ANALYSIS FUNCTION
# ============================================

def analyze_stock(stock_name):
    stock_name = stock_name.upper().strip()

    if stock_name not in nifty50_map:
        print("❌ Stock not found in Nifty50 map. Check the name.")
        return

    ticker = nifty50_map[stock_name]["ticker"]

    print(f"\nAnalyzing {stock_name} ({ticker})")
    print("-" * 60)

    # 1. Latest price data + technicals
    try:
        df = get_price_data(ticker, years=2)
        df = add_technical_indicators(df)
    except Exception as e:
        print(f"⚠️ Error fetching or processing price data: {e}")
        return

    latest = df.iloc[-1]

    last_close = to_scalar(latest['Close'])
    ma20 = to_scalar(latest['MA20'])
    ma50 = to_scalar(latest['MA50'])
    ma200 = to_scalar(latest['MA200'])
    rsi14 = to_scalar(latest['RSI14'])

    # Live / near-live price
    live_price = get_live_price(ticker)

    print(f"Last EOD Close (daily data): {last_close:.2f}")
    if live_price is not None:
        print(f"Live / Latest Quote      : {live_price:.2f}")
    else:
        print("Live / Latest Quote      : Not available")

    print(f"MA20: {ma20:.2f}, MA50: {ma50:.2f}, MA200: {ma200:.2f}")
    print(f"RSI14: {rsi14:.2f}")

    tech_score, tech_reasons = score_technicals(df)

    # 2. Latest fundamentals
    fundamentals = get_fundamentals_yfinance(ticker)

    print("\nFundamentals (yfinance, live-ish):")
    print(f"PE: {fundamentals['pe']}")
    print(f"ROE (%): {fundamentals['roe']}")
    print(f"Debt/Equity: {fundamentals['de_ratio']}")

    fund_score, fund_reasons = score_fundamentals(fundamentals)

    # 3. Final recommendation
    rec, total_score = final_recommendation(tech_score, fund_score)

    print("\n=== RECOMMENDATION ===")
    print(f"Overall Score: {total_score} (Tech: {tech_score}, Fundamental: {fund_score})")
    print(f"Final View: {rec}")

    print("\nReasons (Technicals):")
    for r in tech_reasons:
        print(f"- {r}")

    print("\nReasons (Fundamentals):")
    for r in fund_reasons:
        print(f"- {r}")

# ============================================
# 9. INTERACTIVE DROPDOWN UI
# ============================================

def interactive_analyzer(stock_name):
    analyze_stock(stock_name)

dropdown = Dropdown(
    options=sorted(nifty50_map.keys()),
    description='Stock:',
    disabled=False,
)

_ = interact(interactive_analyzer, stock_name=dropdown)


interactive(children=(Dropdown(description='Stock:', options=('ADANIENT', 'ADANIPORTS', 'APOLLOHOSP', 'ASIANPA…